<a href="https://colab.research.google.com/github/Modequick/Machine-Learning-Project/blob/main/Stock_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
#importing

import yfinance as yf 
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import json

### Creating Dataset to do Analysis

In [243]:
dim_saham = pd.read_excel("/content/drive/MyDrive/Dataset/Daftar Saham.xlsx")
dim_saham = dim_saham[dim_saham["Kode"]!= "ELPI"]

In [244]:
dim_saham["Papan Pencatatan"].value_counts()

Pengembangan    441
Utama           362
Akselerasi       33
PENGEMBANGAN     16
Ekonomi Baru      3
UTAMA             2
Name: Papan Pencatatan, dtype: int64

In [245]:
indo_stock = []
for i in (dim_saham[dim_saham["Papan Pencatatan"]=="Utama"]["Kode"].astype(str)+".JK") :
  data =  yf.download(i,start = "2017-04-01", end  = datetime.datetime.now().strftime("%Y-%m-%d")) 
  data["Stock Code"] = i 
  indo_stock.append(data)
fact_stock_price = pd.concat(indo_stock)  
fact_stock_price.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Open,High,Low,Close,Adj Close,Volume,Stock Code
Date,,,,,,,
2023-02-24,925.0,925.0,815.0,875.0,875.0,584706700,PGEO.JK
2023-02-27,875.0,875.0,830.0,830.0,830.0,90183300,PGEO.JK
2023-02-28,815.0,840.0,800.0,800.0,800.0,65393800,PGEO.JK
2023-03-01,800.0,840.0,800.0,835.0,835.0,43259400,PGEO.JK
2023-03-02,840.0,875.0,820.0,870.0,870.0,55298000,PGEO.JK


In [246]:
interesting_columns = [
    "industry",
    "sector",
    # "fullTimeEmployees",
    # "previousClose",
    # "dividendRate",
    # "exDividendDate",
    # "payoutRatio",
    "marketCap"
]  
fact_stock_market_cap = []
for i in (dim_saham[dim_saham["Papan Pencatatan"]=="Utama"]["Kode"].head(355).astype(str)+".JK") : 
    ticker = yf.Ticker(i)
    data = ticker.info
    get_data = pd.DataFrame.from_dict(data)
    clean_data = get_data[interesting_columns].drop_duplicates()
    clean_data["Stock Code"] = i
    fact_stock_market_cap.append(clean_data)

fact_stock_market_cap  = pd.concat(fact_stock_market_cap)
fact_stock_market_cap

,industry,sector,marketCap,Stock Code
0,Utilities—Renewable,Utilities,31254055485440,PGEO.JK
0,Other Industrial Metals & Mining,Basic Materials,8048858890240,HILL.JK
0,Household & Personal Products,Consumer Defensive,1226508730368,TCID.JK
0,Other Industrial Metals & Mining,Basic Materials,7559466450944,TINS.JK
0,Other Industrial Metals & Mining,Basic Materials,11241899687936,CUAN.JK
...,...,...,...,...
0,Specialty Chemicals,Basic Materials,37172161806336,AVIA.JK
0,Auto Parts,Consumer Cyclical,4823527194624,DRMA.JK
0,Farm Products,Consumer Defensive,10903400480768,STAA.JK
0,Farm Products,Consumer Defensive,6162533580800,TLDN.JK


### Final Dataset

In [257]:
#Final
dim_saham 
fact_stock_price
fact_stock_market_cap 


#adding dense_rank 
fact_stock_market_cap["Rank"] = fact_stock_market_cap["marketCap"].rank(method='dense', ascending=False)
fact_stock_market_cap

,industry,sector,marketCap,Stock Code,Ramk,Rank
0,Utilities—Renewable,Utilities,31254055485440,PGEO.JK,47.0,47.0
0,Other Industrial Metals & Mining,Basic Materials,8048858890240,HILL.JK,112.0,112.0
0,Household & Personal Products,Consumer Defensive,1226508730368,TCID.JK,260.0,260.0
0,Other Industrial Metals & Mining,Basic Materials,7559466450944,TINS.JK,121.0,121.0
0,Other Industrial Metals & Mining,Basic Materials,11241899687936,CUAN.JK,92.0,92.0
...,...,...,...,...,...,...
0,Specialty Chemicals,Basic Materials,37172161806336,AVIA.JK,43.0,43.0
0,Auto Parts,Consumer Cyclical,4823527194624,DRMA.JK,149.0,149.0
0,Farm Products,Consumer Defensive,10903400480768,STAA.JK,94.0,94.0
0,Farm Products,Consumer Defensive,6162533580800,TLDN.JK,136.0,136.0


In [261]:
df = pd.merge(fact_stock_price,fact_stock_market_cap[["Stock Code","Rank","sector","marketCap"]], on = "Stock Code", how = "left")
df.head(1)

,Open,High,Low,Close,Adj Close,Volume,Stock Code,Rank,sector,marketCap
0,925.0,925.0,815.0,875.0,875.0,584706700,PGEO.JK,47.0,Utilities,3.125406e+13


In [264]:
df.sort_values(by = "Rank", ascending = True)

,Open,High,Low,Close,Adj Close,Volume,Stock Code,Rank,sector,marketCap
61177,4770.0,4795.0,4770.0,4775.0,4355.914062,67938000,BBCA.JK,1.0,Financial Services,1.115639e+15
61838,6315.0,6320.0,6145.0,6200.0,5943.523926,116961000,BBCA.JK,1.0,Financial Services,1.115639e+15
61837,6200.0,6215.0,6125.0,6135.0,5881.212402,123697500,BBCA.JK,1.0,Financial Services,1.115639e+15
61836,6155.0,6275.0,6135.0,6250.0,5908.629883,107908000,BBCA.JK,1.0,Financial Services,1.115639e+15
61835,6135.0,6170.0,6080.0,6165.0,5828.272461,80215000,BBCA.JK,1.0,Financial Services,1.115639e+15
...,...,...,...,...,...,...,...,...,...,...
483313,296.0,300.0,288.0,288.0,288.000000,617700,SUNI.JK,NaN,NaN,NaN
483314,292.0,292.0,282.0,284.0,284.000000,377500,SUNI.JK,NaN,NaN,NaN
483315,288.0,288.0,276.0,282.0,282.000000,478200,SUNI.JK,NaN,NaN,NaN
483316,282.0,296.0,282.0,284.0,284.000000,655700,SUNI.JK,NaN,NaN,NaN


In [267]:
df["Close"].rolling(window = 14).mean()

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
             ...    
483313    293.857143
483314    293.142857
483315    292.428571
483316    291.857143
483317    291.142857
Name: Close, Length: 483318, dtype: float64